## Put all imports necessary here

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt

## Load the Data

In [8]:
# Load the training data
train_file = np.load('train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)

# Load the testing data
test_file = np.load('test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


## Data Preprocessing

In [9]:
def prepare_data(data, is_train=True):
    """Extract and reshape for LSTM (seq_len, batch, features)"""
    if is_train:
        X = data[:, 0, :50, :]  # (N, 50, 6)
        Y = data[:, 0, 50:, :2]  # (N, 60, 2)
    else:
        X = data[:, 0, :, :]    # (N, 50, 6)
        Y = None
    
    # Normalize
    pos_mean, pos_std = X[..., :2].mean(), X[..., :2].std()
    X_norm = (X - pos_mean) / pos_std
    if Y is not None:
        Y_norm = (Y - pos_mean) / pos_std
    else:
        Y_norm = None
    
    # Reshape for LSTM: (seq_len, batch, features)
    X_norm = np.swapaxes(X_norm, 0, 1)  # (50, N, 6)
    if Y is not None:
        Y_norm = np.swapaxes(Y_norm, 0, 1)  # (60, N, 2)
    
    return X_norm, Y_norm, (pos_mean, pos_std)

X_train, Y_train, norm_stats = prepare_data(train_data)
X_test, _, _ = prepare_data(test_data, is_train=False)

## Define Models

In [10]:
class TrajectoryLSTM(nn.Module):
    def __init__(self, input_size=6, hidden_size=128):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,
            dropout=0.2,
            batch_first=False
        )
        self.fc = nn.Linear(hidden_size, 2)
        
    def forward(self, x, pred_steps=60):
        # Process input sequence (50 steps)
        _, (hidden, cell) = self.lstm(x)  # x: (50, N, 6)
        
        # Initialize predictions
        predictions = []
        last_input = x[-1:, :, :]  # (1, N, 6)
        
        # Autoregressive prediction for 60 steps
        for _ in range(pred_steps):
            lstm_out, (hidden, cell) = self.lstm(
                last_input, 
                (hidden, cell)  # Correct: pass the previous hidden/cell states
            )
            pred = self.fc(lstm_out)  # (1, N, 2)
            predictions.append(pred)
            
            # Create next input with prediction + zeros for other features
            last_input = torch.cat([
                pred,
                torch.zeros_like(last_input[:, :, 2:])
            ], dim=-1)
        
        return torch.cat(predictions, dim=0)  # (60, N, 2)

model = TrajectoryLSTM()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

## Do Training Loop

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)  # (50, 10000, 6)
Y_train_tensor = torch.FloatTensor(Y_train).to(device)  # (60, 10000, 2)

for epoch in range(15):  # LSTMs often need more epochs
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_train_tensor)  # (60, 10000, 2)
    
    # Calculate loss (only compare with actual future steps)
    loss = criterion(outputs, Y_train_tensor)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 1.0023
Epoch 2, Loss: 0.9820
Epoch 3, Loss: 0.9638
Epoch 4, Loss: 0.9473
Epoch 5, Loss: 0.9322
Epoch 6, Loss: 0.9190
Epoch 7, Loss: 0.9095
Epoch 8, Loss: 0.9079
Epoch 9, Loss: 0.9152
Epoch 10, Loss: 0.9153
Epoch 11, Loss: 0.9093
Epoch 12, Loss: 0.9037
Epoch 13, Loss: 0.9003
Epoch 14, Loss: 0.8983
Epoch 15, Loss: 0.8964


## Predict and Create Submission

In [12]:
model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test).to(device)  # (50, 2100, 6)
    preds_norm = model(X_test_tensor).cpu().numpy()  # (60, 2100, 2)

# Reshape and denormalize
preds_norm = np.swapaxes(preds_norm, 0, 1)  # (2100, 60, 2)
preds = preds_norm * norm_stats[1] + norm_stats[0]

# Create submission
submission = preds.reshape(-1, 2)  # (126000, 2)
submission_df = pd.DataFrame(submission, columns=['x', 'y'])
submission_df.index.name = 'index'
submission_df.to_csv('lstm_submission.csv')

print("LSTM Submission saved!")

LSTM Submission saved!
